In [ ]:
# =============================================================================
# CELL 1: ENVIRONMENT SETUP (RUN FIRST, THEN RESTART RUNTIME!)
# =============================================================================
# IMPORTANT: Run this cell ONCE, then go to the menu and select
# "Runtime" -> "Disconnect and delete runtime" before running anything else.

print("Performing a full environment setup to ensure library compatibility...")

# Step 1: Uninstall potentially conflicting or broken libraries
!pip uninstall -y albumentations albucore torch torchvision opencv-python opencv-python-headless --quiet

# Step 2: Install compatible PyTorch and Torchvision for a specific CUDA version
print("Installing compatible Torch and Torchvision...")
!pip install torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu118 --quiet

# Step 3: Install our other pinned dependencies
print("Installing other required libraries...")
!pip install numpy==1.26.4 --quiet
!pip install opencv-python-headless==4.8.0.74 --quiet
!pip install imageio-ffmpeg --quiet
!pip install scikit-learn --quiet
!pip install tqdm --quiet
!pip install matplotlib --quiet

print("\n Full environment setup complete.")
print("  STOP HERE! Go to Runtime -> Restart Runtime, then continue with next cells.")

Performing a full environment setup to ensure library compatibility...
Installing compatible Torch and Torchvision...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 434.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 65.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 19.6 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires opencv-python-headless, which is not installed.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.1.0+cu118 which is incompatible.
Installing other required libraries...
Reason for being yanked: deprecated, use 4.8.0.76
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 2.7 MB/s eta 0:00:000:00:0100:01

✅ Full environment setup complete.
⚠️  STOP HERE! Go to Runtime -> Restart Runtime, t

In [ ]:
# =============================================================================
# CELL 2: VERIFY INSTALLATION (RUN AFTER RESTART)
# =============================================================================

print("Verifying installed packages...")
import torch
import torchvision
import cv2
import numpy as np
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ Torchvision: {torchvision.__version__}")
print(f"✅ OpenCV: {cv2.__version__}")
print(f"✅ NumPy: {np.__version__}")
print(f"✅ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f" CUDA Version: {torch.version.cuda}")
print("\n All libraries verified! Ready to train.")

In [ ]:
# =============================================================================
# CELL 3: IMPORTS & CONFIGURATION
# =============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from pathlib import Path
from typing import List, Tuple
import cv2
import numpy as np
import random
from tqdm import tqdm
import time

CONFIG = {
    'dataset_path': Path("/kaggle/input/celeb-df-v2/"),
    'subset_size': None,
    'batch_size': 4,
    'epochs': 50,
    'learning_rate': 1e-4,     # Back to a standard rate
    'weight_decay': 1e-4,
    'frame_count': 16,
    'image_size': 112,
    'patience': 10,
    'num_workers': 2,
    'save_dir': Path('/kaggle/working/celeb_df_results_resnet_baseline'), # New name
    'seed': 42,
    'warmup_epochs': 3, # Doesn't matter since AFSL is off, but good to reset
}

print("✅ Configuration loaded")

✅ Configuration loaded


In [2]:
# =============================================================================
# CELL 4: HELPER FUNCTIONS
# =============================================================================

def set_seed(seed: int):
    """Sets the random seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def get_device() -> torch.device:
    """Returns the appropriate device (CUDA or CPU)."""
    if torch.cuda.is_available():
        print("✅ Using CUDA GPU for training.")
        return torch.device("cuda")
    else:
        print("⚠️ CUDA not available. Using CPU for training.")
        return torch.device("cpu")

print("✅ Helper functions defined")

✅ Helper functions defined


In [3]:
# =============================================================================
# CELL 5: ADVERSARIAL ATTACK
# =============================================================================

def pgd_attack(model, inputs, labels, epsilon=4/255, alpha=1/255, num_iter=3):
    """
    Creates adversarial examples using PGD attack.
    Reduced strength to prevent overwhelming the model.
    """
    original_inputs = inputs.clone().detach()
    adv_inputs = inputs.clone().detach()
    adv_inputs.requires_grad = True
    criterion = nn.CrossEntropyLoss()

    for _ in range(num_iter):
        model.zero_grad()
        _, outputs = model(adv_inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        adv_inputs = adv_inputs.detach() + alpha * adv_inputs.grad.sign()
        perturbation = torch.clamp(adv_inputs - original_inputs, min=-epsilon, max=epsilon)
        adv_inputs = torch.clamp(original_inputs + perturbation, min=-10, max=10).detach()
        adv_inputs.requires_grad = True

    return adv_inputs

print("✅ Adversarial attack function defined")

✅ Adversarial attack function defined


In [4]:
# =============================================================================
# CELL 6: DATA LOADING
# =============================================================================

def load_video_paths(dataset_root: Path) -> Tuple[List[Path], List[Path]]:
    """Loads video file paths from the Celeb-DF directory structure."""
    real_videos = list((dataset_root / "Celeb-real").glob("*.mp4"))
    fake_videos = list((dataset_root / "Celeb-synthesis").glob("*.mp4"))
    print(f"Found {len(real_videos):,} REAL videos and {len(fake_videos):,} FAKE videos.")
    print(f"Class distribution: REAL={len(real_videos)/(len(real_videos)+len(fake_videos))*100:.1f}%, "
          f"FAKE={len(fake_videos)/(len(real_videos)+len(fake_videos))*100:.1f}%")
    return real_videos, fake_videos

class CelebDFDataset(Dataset):
    """PyTorch Dataset for loading and processing video frames."""
    def __init__(self, video_paths: List[Path], labels: List[int], transform=None, frame_count: int = 16):
        self.video_paths = video_paths
        self.labels = labels
        self.transform = transform
        self.frame_count = frame_count
        self.failed_videos = 0

    def __len__(self) -> int:
        return len(self.video_paths)

    def _sample_frames(self, video_path: Path) -> np.ndarray:
        cap = cv2.VideoCapture(str(video_path))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if not cap.isOpened() or total_frames < self.frame_count:
            if cap.isOpened(): cap.release()
            self.failed_videos += 1
            return np.zeros((self.frame_count, CONFIG['image_size'], CONFIG['image_size'], 3), dtype=np.uint8)

        indices = np.linspace(0, total_frames - 1, self.frame_count, dtype=int)
        frames = []
        for idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (CONFIG['image_size'], CONFIG['image_size']), interpolation=cv2.INTER_AREA)
                frames.append(frame)
            else:
                frames.append(np.zeros((CONFIG['image_size'], CONFIG['image_size'], 3), dtype=np.uint8))
        cap.release()
        return np.array(frames)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        video_path = self.video_paths[idx]
        label = self.labels[idx]
        frames_np = self._sample_frames(video_path)
        
        if self.transform:
            frames = torch.stack([self.transform(frame) for frame in frames_np])
            
        return frames, label

print("✅ Data loading classes defined")

✅ Data loading classes defined


In [5]:
# =============================================================================
# CELL 7: MODEL ARCHITECTURE
# =============================================================================

class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bn2(self.pointwise(self.relu(self.bn1(self.depthwise(x))))))

class LightweightDeepfakeDetector(nn.Module):
    def __init__(self, num_classes=2, dropout_rate=0.6):
        super().__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(inplace=True),
            DepthwiseSeparableConv(32, 64, stride=1),
            DepthwiseSeparableConv(64, 128, stride=2),
            DepthwiseSeparableConv(128, 256, stride=2),
            DepthwiseSeparableConv(256, 512, stride=2),
        )
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.temporal_module = nn.LSTM(512, 128, batch_first=True, bidirectional=True)
        self.classifier = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(inplace=True), 
            nn.Dropout(dropout_rate),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        b, t, c, h, w = x.shape
        x = x.view(b * t, c, h, w)
        features = self.backbone(x)
        features = self.global_pool(features).view(b, t, -1)
        lstm_out, _ = self.temporal_module(features)
        temporal_features = torch.cat((lstm_out[:, -1, :128], lstm_out[:, 0, 128:]), dim=-1)
        output = self.classifier(temporal_features)
        return temporal_features, output

print("✅ Model architecture defined")

✅ Model architecture defined


In [ ]:
# =============================================================================
# CELL 8: TRAINING LOOP
# =============================================================================

def train_model(model, train_loader, val_loader, optimizer, scheduler, device, epochs, save_dir):
    """Enhanced training loop with warmup and better loss balancing."""
    save_dir.mkdir(parents=True, exist_ok=True)
    best_val_acc = 0.0
    patience_counter = 0

    classification_criterion = nn.CrossEntropyLoss()
    similarity_criterion = nn.CosineSimilarity(dim=1)
    
    # AFSL hyperparameters (activated after warmup)
    beta1 = 0.5   # Reduced from 1.0
    beta2 = 0.05  # Reduced from 0.1

    for epoch in range(epochs):
        print(f"\n{'='*60}")
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"{'='*60}")
        
        # Determine if we should use AFSL
        use_afsl = False  # <-- TEMPORARILY DISABLE AFSL
        if epoch == CONFIG['warmup_epochs']:
            print(" Warmup complete! Activating AFSL...")
        
        # --- Training phase ---
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        all_preds, all_labels = [], []
        
        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            clean_features, clean_outputs = model(inputs)
            loss_dcl = classification_criterion(clean_outputs, labels)
            
            if use_afsl:
                adv_inputs = pgd_attack(model, inputs, labels)
                adv_features, _ = model(adv_inputs)
                
                loss_asl = (1 - similarity_criterion(clean_features, adv_features)).mean()
                
                real_mask = labels == 0
                fake_mask = labels == 1
                loss_srl = 0.0
                if real_mask.sum() > 0 and fake_mask.sum() > 0:
                    real_features = clean_features[real_mask]
                    fake_features = clean_features[fake_mask]
                    loss_srl = similarity_criterion(
                        real_features.mean(dim=0, keepdim=True), 
                        fake_features.mean(dim=0, keepdim=True)
                    ).mean()

                total_loss = loss_dcl + (beta1 * loss_asl) + (beta2 * loss_srl)
            else:
                total_loss = loss_dcl
            
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += total_loss.item() * inputs.size(0)
            _, preds = torch.max(clean_outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
        # --- Validation phase ---
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_preds, val_labels = [], []
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                _, outputs = model(inputs)
                loss = classification_criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        # --- Metrics ---
        train_accuracy = 100 * train_correct / train_total
        avg_train_loss = train_loss / train_total
        val_accuracy = 100 * val_correct / val_total
        avg_val_loss = val_loss / val_total
        
        train_cm = confusion_matrix(all_labels, all_preds)
        val_cm = confusion_matrix(val_labels, val_preds)
        
        print(f"\n📊 Training:")
        print(f"   Loss: {avg_train_loss:.4f} | Acc: {train_accuracy:.2f}%")
        print(f"   Confusion Matrix:\n{train_cm}")
        
        print(f"\n📊 Validation:")
        print(f"   Loss: {avg_val_loss:.4f} | Acc: {val_accuracy:.2f}%")
        print(f"   Confusion Matrix:\n{val_cm}")

        scheduler.step(val_accuracy)

        if val_accuracy > best_val_acc:
            best_val_acc, patience_counter = val_accuracy, 0
            torch.save(model.state_dict(), save_dir / "best_model.pth")
            print(f"\n Best model saved: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            print(f"\n Patience: {patience_counter}/{CONFIG['patience']}")
            if patience_counter >= CONFIG['patience']:
                print(" Early stopping")
                break

print("✅ Training function defined")

✅ Training function defined


In [ ]:
# =============================================================================
# CELL 9: MAIN FUNCTION
# =============================================================================

def main():
    set_seed(CONFIG['seed'])
    device = get_device()
    
    # Load data (THE FULL DATASET)
    real_videos, fake_videos = load_video_paths(CONFIG['dataset_path'])
    all_videos = real_videos + fake_videos
    all_labels = [0] * len(real_videos) + [1] * len(fake_videos)
    print(f"\n📋 Labels: 0=REAL, 1=FAKE")

    # Split (THE FULL DATASET)
    train_videos, val_videos, train_labels, val_labels = train_test_split(
        all_videos, all_labels, test_size=0.2, stratify=all_labels, random_state=CONFIG['seed']
    )

    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    val_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    train_dataset = CelebDFDataset(train_videos, train_labels, transform=train_transform, frame_count=CONFIG['frame_count'])
    val_dataset = CelebDFDataset(val_videos, val_labels, transform=val_transform, frame_count=CONFIG['frame_count'])

    # Balanced sampler
    class_counts = np.bincount(train_labels)
    class_weights = 1. / class_counts
    sample_weights = np.array([class_weights[label] for label in train_labels])
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], sampler=sampler, num_workers=CONFIG['num_workers'], pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=CONFIG['num_workers'], pin_memory=True)

    # Model setup
    model = LightweightDeepfakeDetector().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
    
    # Train
    print(f"\n Starting training...")
    start_time = time.time()
    train_model(model, train_loader, val_loader, optimizer, scheduler, device, CONFIG['epochs'], CONFIG['save_dir'])
    end_time = time.time()
    print(f"\n✅ Done in {(end_time - start_time) / 60:.2f} minutes.")
    
    if train_dataset.failed_videos > 0:
        print(f" {train_dataset.failed_videos} videos failed to load.")

print("✅ Main function defined")

✅ Main function defined


In [ ]:
# =============================================================================
# ADD THIS AS A NEW CELL - FOCAL LOSS & IMPROVED TRAINING
# =============================================================================

from sklearn.metrics import f1_score

class FocalLoss(nn.Module):
    """Focal Loss for handling class imbalance."""
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

def train_model_focal(model, train_loader, val_loader, optimizer, scheduler, device, epochs, save_dir):
    """Training with Focal Loss and per-class metrics."""
    save_dir.mkdir(parents=True, exist_ok=True)
    best_val_f1 = 0.0
    patience_counter = 0

    # Focal Loss with higher alpha for minority class
    classification_criterion = FocalLoss(alpha=0.75, gamma=2.0)
    similarity_criterion = nn.CosineSimilarity(dim=1)
    
    beta1 = 0.3
    beta2 = 0.03

    for epoch in range(epochs):
        print(f"\n{'='*70}")
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"{'='*70}")
        
        use_afsl = epoch >= CONFIG['warmup_epochs']
        if epoch == CONFIG['warmup_epochs']:
            print(" Warmup complete! Activating AFSL...")
        
        # Training
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        all_preds, all_labels = [], []
        
        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            
            clean_features, clean_outputs = model(inputs)
            loss_dcl = classification_criterion(clean_outputs, labels)
            
            if use_afsl:
                adv_inputs = pgd_attack(model, inputs, labels)
                adv_features, _ = model(adv_inputs)
                loss_asl = (1 - similarity_criterion(clean_features, adv_features)).mean()
                
                real_mask = labels == 0
                fake_mask = labels == 1
                loss_srl = 0.0
                if real_mask.sum() > 0 and fake_mask.sum() > 0:
                    real_features = clean_features[real_mask]
                    fake_features = clean_features[fake_mask]
                    loss_srl = similarity_criterion(
                        real_features.mean(dim=0, keepdim=True), 
                        fake_features.mean(dim=0, keepdim=True)
                    ).mean()
                total_loss = loss_dcl + (beta1 * loss_asl) + (beta2 * loss_srl)
            else:
                total_loss = loss_dcl
            
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += total_loss.item() * inputs.size(0)
            _, preds = torch.max(clean_outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_preds, val_labels = [], []
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                _, outputs = model(inputs)
                loss = classification_criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        # Metrics
        train_accuracy = 100 * train_correct / train_total
        avg_train_loss = train_loss / train_total
        val_accuracy = 100 * val_correct / val_total
        avg_val_loss = val_loss / val_total
        
        train_f1 = f1_score(all_labels, all_preds, average='macro')
        val_f1 = f1_score(val_labels, val_preds, average='macro')
        
        train_cm = confusion_matrix(all_labels, all_preds)
        val_cm = confusion_matrix(val_labels, val_preds)
        
        train_real_acc = 100 * train_cm[0, 0] / train_cm[0].sum() if train_cm[0].sum() > 0 else 0
        train_fake_acc = 100 * train_cm[1, 1] / train_cm[1].sum() if train_cm[1].sum() > 0 else 0
        val_real_acc = 100 * val_cm[0, 0] / val_cm[0].sum() if val_cm[0].sum() > 0 else 0
        val_fake_acc = 100 * val_cm[1, 1] / val_cm[1].sum() if val_cm[1].sum() > 0 else 0
        
        print(f"\n Training:")
        print(f"   Overall - Loss: {avg_train_loss:.4f} | Acc: {train_accuracy:.2f}% | F1: {train_f1:.4f}")
        print(f"   Per-Class - REAL: {train_real_acc:.2f}% | FAKE: {train_fake_acc:.2f}%")
        print(f"   Confusion Matrix:\n{train_cm}")
        
        print(f"\n Validation:")
        print(f"   Overall - Loss: {avg_val_loss:.4f} | Acc: {val_accuracy:.2f}% | F1: {val_f1:.4f}")
        print(f"   Per-Class - REAL: {val_real_acc:.2f}% | FAKE: {val_fake_acc:.2f}%")
        print(f"   Confusion Matrix:\n{val_cm}")

        scheduler.step(val_f1)

        if val_f1 > best_val_f1:
            best_val_f1, patience_counter = val_f1, 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_f1': val_f1,
                'val_accuracy': val_accuracy,
            }, save_dir / "best_model_focal.pth")
            print(f"\n🎉 Best F1: {best_val_f1:.4f} (Acc: {val_accuracy:.2f}%)")
        else:
            patience_counter += 1
            print(f"\n Patience: {patience_counter}/{CONFIG['patience']}")
            if patience_counter >= CONFIG['patience']:
                print(" Early stopping")
                break

print("✅ Focal Loss training function ready!")

# =============================================================================
# RUN THIS CELL TO START FOCAL LOSS TRAINING
# =============================================================================

def main_with_focal():
    """Run training with Focal Loss."""
    set_seed(CONFIG['seed'])
    device = get_device()
    
    # Load data
    real_videos, fake_videos = load_video_paths(CONFIG['dataset_path'])
    all_videos = real_videos + fake_videos
    all_labels = [0] * len(real_videos) + [1] * len(fake_videos)
    print(f"\n📋 Labels: 0=REAL, 1=FAKE")

    # Split
    train_videos, val_videos, train_labels, val_labels = train_test_split(
        all_videos, all_labels, test_size=0.2, stratify=all_labels, random_state=CONFIG['seed']
    )

    # Transforms
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    val_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    train_dataset = CelebDFDataset(train_videos, train_labels, transform=train_transform, frame_count=CONFIG['frame_count'])
    val_dataset = CelebDFDataset(val_videos, val_labels, transform=val_transform, frame_count=CONFIG['frame_count'])

    # Balanced sampler
    class_counts = np.bincount(train_labels)
    class_weights = 1. / class_counts
    sample_weights = np.array([class_weights[label] for label in train_labels])
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], sampler=sampler, num_workers=CONFIG['num_workers'], pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=CONFIG['num_workers'], pin_memory=True)

    # Model
    model = LightweightDeepfakeDetector().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
    
    # Train with Focal Loss
    print(f"\n🚀 Starting training with Focal Loss (alpha=0.75, gamma=2.0)...")
    start_time = time.time()
    train_model_focal(model, train_loader, val_loader, optimizer, scheduler, device, CONFIG['epochs'], CONFIG['save_dir'])
    end_time = time.time()
    print(f"\n✅ Training complete in {(end_time - start_time) / 60:.2f} minutes.")

✅ Focal Loss training function ready!


In [ ]:
# =============================================================================
# FINAL BEST PRACTICES TRAINING - All Improvements Combined
# =============================================================================

from sklearn.metrics import f1_score, classification_report

def train_model_best_practices(model, train_loader, val_loader, optimizer, scheduler, device, epochs, save_dir):
    """
    Final optimized training with all best practices.
    """
    save_dir.mkdir(parents=True, exist_ok=True)
    best_val_f1 = 0.0
    patience_counter = 0
    
    # IMPROVEMENT 1: Balanced weight for minority REAL class
    class_weights = torch.tensor([2.0, 1.0]).to(device)  # Sweet spot for 30-70 split
    classification_criterion = nn.CrossEntropyLoss(weight=class_weights)
    
    # Track history for analysis
    history = {
        'train_loss': [], 'train_acc': [], 'train_f1': [],
        'val_loss': [], 'val_acc': [], 'val_f1': [],
        'train_real_acc': [], 'train_fake_acc': [],
        'val_real_acc': [], 'val_fake_acc': []
    }

    for epoch in range(epochs):
        print(f"\n{'='*70}")
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"{'='*70}")
        
        # --- Training phase ---
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        all_preds, all_labels = [], []
        
        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            
            _, outputs = model(inputs)
            loss = classification_criterion(outputs, labels)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
        # --- Validation phase ---
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_preds, val_labels = [], []
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation"):
                inputs, labels = inputs.to(device), labels.to(device)
                _, outputs = model(inputs)
                loss = classification_criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        # --- Calculate metrics ---
        train_accuracy = 100 * train_correct / train_total
        avg_train_loss = train_loss / train_total
        val_accuracy = 100 * val_correct / val_total
        avg_val_loss = val_loss / val_total
        
        train_f1 = f1_score(all_labels, all_preds, average='macro')
        val_f1 = f1_score(val_labels, val_preds, average='macro')
        
        train_cm = confusion_matrix(all_labels, all_preds)
        val_cm = confusion_matrix(val_labels, val_preds)
        
        train_real_acc = 100 * train_cm[0, 0] / train_cm[0].sum() if train_cm[0].sum() > 0 else 0
        train_fake_acc = 100 * train_cm[1, 1] / train_cm[1].sum() if train_cm[1].sum() > 0 else 0
        val_real_acc = 100 * val_cm[0, 0] / val_cm[0].sum() if val_cm[0].sum() > 0 else 0
        val_fake_acc = 100 * val_cm[1, 1] / val_cm[1].sum() if val_cm[1].sum() > 0 else 0
        
        # Store history
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['train_f1'].append(train_f1)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)
        history['val_f1'].append(val_f1)
        history['train_real_acc'].append(train_real_acc)
        history['train_fake_acc'].append(train_fake_acc)
        history['val_real_acc'].append(val_real_acc)
        history['val_fake_acc'].append(val_fake_acc)
        
        # --- Display results ---
        print(f"\n📊 Training:")
        print(f"   Overall - Loss: {avg_train_loss:.4f} | Acc: {train_accuracy:.2f}% | F1: {train_f1:.4f}")
        print(f"   Per-Class - REAL: {train_real_acc:.2f}% | FAKE: {train_fake_acc:.2f}%")
        print(f"   Confusion Matrix:")
        print(f"   [[{train_cm[0, 0]:4d} {train_cm[0, 1]:4d}]   ← REAL predictions")
        print(f"    [{train_cm[1, 0]:4d} {train_cm[1, 1]:4d}]]  ← FAKE predictions")
        
        print(f"\n📊 Validation:")
        print(f"   Overall - Loss: {avg_val_loss:.4f} | Acc: {val_accuracy:.2f}% | F1: {val_f1:.4f}")
        print(f"   Per-Class - REAL: {val_real_acc:.2f}% | FAKE: {val_fake_acc:.2f}%")
        print(f"   Confusion Matrix:")
        print(f"   [[{val_cm[0, 0]:4d} {val_cm[0, 1]:4d}]   ← REAL predictions")
        print(f"    [{val_cm[1, 0]:4d} {val_cm[1, 1]:4d}]]  ← FAKE predictions")

        scheduler.step(val_f1)

        # --- Save best model ---
        if val_f1 > best_val_f1:
            best_val_f1, patience_counter = val_f1, 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_f1': val_f1,
                'val_accuracy': val_accuracy,
                'history': history,
                'val_cm': val_cm,
            }, save_dir / "best_model_final.pth")
            print(f"\n Best F1: {best_val_f1:.4f} (Acc: {val_accuracy:.2f}%)")
            print(f"   REAL: {val_real_acc:.2f}% | FAKE: {val_fake_acc:.2f}%")
        else:
            patience_counter += 1
            print(f"\n Patience: {patience_counter}/{CONFIG['patience']}")
            if patience_counter >= CONFIG['patience']:
                print(" Early stopping")
                break
    
    # --- Final report ---
    print(f"\n{'='*70}")
    print("📈 TRAINING SUMMARY")
    print(f"{'='*70}")
    print(f"Best Validation F1: {best_val_f1:.4f}")
    print(f"Best Validation Accuracy: {max(history['val_acc']):.2f}%")
    print(f"Best REAL Accuracy: {max(history['val_real_acc']):.2f}%")
    print(f"Best FAKE Accuracy: {max(history['val_fake_acc']):.2f}%")
    
    return history

print("✅ Best practices training function ready!")

# =============================================================================
# IMPROVED MODEL WITH HIGHER DROPOUT
# =============================================================================

class ImprovedDeepfakeDetector(nn.Module):
    """Enhanced model with higher dropout to reduce overfitting."""
    def __init__(self, num_classes=2, dropout_rate=0.6): 
        super().__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(inplace=True),
            DepthwiseSeparableConv(32, 64, stride=1),
            DepthwiseSeparableConv(64, 128, stride=2),
            DepthwiseSeparableConv(128, 256, stride=2),
            DepthwiseSeparableConv(256, 512, stride=2),
        )
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.temporal_module = nn.LSTM(512, 128, batch_first=True, bidirectional=True)
        self.classifier = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(inplace=True), 
            nn.Dropout(dropout_rate),  # Higher dropout
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        b, t, c, h, w = x.shape
        x = x.view(b * t, c, h, w)
        features = self.backbone(x)
        features = self.global_pool(features).view(b, t, -1)
        lstm_out, _ = self.temporal_module(features)
        temporal_features = torch.cat((lstm_out[:, -1, :128], lstm_out[:, 0, 128:]), dim=-1)
        output = self.classifier(temporal_features)
        return temporal_features, output

print("✅ Improved model with higher dropout ready!")

# =============================================================================
# FINAL MAIN FUNCTION WITH ALL IMPROVEMENTS
# =============================================================================

def main_final_best():
    """
    Final training with all best practices:
    - Balanced dataset (30-70 split)
    - Stronger augmentation
    - Higher class weights (3.5x for REAL)
    - Higher dropout (0.6)
    - Lower learning rate (5e-5)
    """
    set_seed(CONFIG['seed'])
    device = get_device()
    
    # Load and balance data
    real_videos, fake_videos = load_video_paths(CONFIG['dataset_path'])
    
    print(f"\n{'='*70}")
    print("🔧 CREATING BALANCED DATASET")
    print(f"{'='*70}")
    
    n_real = len(real_videos)
    target_ratio = 0.30
    n_fake_keep = int(n_real / target_ratio) - n_real
    
    np.random.seed(CONFIG['seed'])
    fake_indices = np.random.choice(len(fake_videos), size=n_fake_keep, replace=False)
    fake_videos_sampled = [fake_videos[i] for i in fake_indices]
    
    all_videos = real_videos + fake_videos_sampled
    all_labels = [0] * len(real_videos) + [1] * len(fake_videos_sampled)
    
    print(f"✅ Balanced dataset: {len(real_videos)} REAL ({30.0:.1f}%), "
          f"{len(fake_videos_sampled)} FAKE ({70.0:.1f}%)")

    # Split
    train_videos, val_videos, train_labels, val_labels = train_test_split(
        all_videos, all_labels, test_size=0.2, stratify=all_labels, random_state=CONFIG['seed']
    )
    
    print(f"Training: {len(train_videos)} | Validation: {len(val_videos)}")

    # IMPROVEMENT 2: Stronger augmentation
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        # Geometric augmentations
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        # Color augmentations
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
        transforms.RandomGrayscale(p=0.1),
        # Normalize
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        # Random erasing
        transforms.RandomErasing(p=0.2, scale=(0.02, 0.1)),
    ])

    val_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    train_dataset = CelebDFDataset(train_videos, train_labels, transform=train_transform, frame_count=CONFIG['frame_count'])
    val_dataset = CelebDFDataset(val_videos, val_labels, transform=val_transform, frame_count=CONFIG['frame_count'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=CONFIG['num_workers'], pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=CONFIG['num_workers'], pin_memory=True)

    # IMPROVEMENT 3: Use improved model with higher dropout
    model = ImprovedDeepfakeDetector(dropout_rate=0.6).to(device)
    
    # IMPROVEMENT 4: Lower learning rate for better generalization
    lr = 5e-5  # Reduced from 1e-4
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=CONFIG['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
    
    print(f"\n🚀 FINAL TRAINING WITH ALL IMPROVEMENTS")
    print(f"{'='*70}")
    print(f"✅ Balanced dataset (30-70 split)")
    print(f"✅ Stronger augmentation (RandomAffine, Grayscale, Erasing)")
    print(f"✅ Higher class weight (REAL=2.0x, FAKE=1.0x)")
    print(f"✅ Higher dropout (0.6)")
    print(f"✅ Lower learning rate ({lr})")
    print(f"{'='*70}\n")
    
    start_time = time.time()
    history = train_model_best_practices(model, train_loader, val_loader, optimizer, scheduler, device, CONFIG['epochs'], CONFIG['save_dir'])
    end_time = time.time()
    
    print(f"\n✅ Training complete in {(end_time - start_time) / 60:.2f} minutes.")
    
    # Save training history
    import json
    with open(CONFIG['save_dir'] / 'training_history.json', 'w') as f:
        json.dump({k: [float(v) if isinstance(v, (int, float, np.number)) else v for v in vals] 
                   for k, vals in history.items()}, f, indent=2)
    print(f"📊 Training history saved to {CONFIG['save_dir'] / 'training_history.json'}")

# =============================================================================
# RUN THE FINAL OPTIMIZED TRAINING!
# ======================================================================

✅ Best practices training function ready!
✅ Improved model with higher dropout ready!


In [10]:
# Step 1: Manually download the weights
print("Downloading weights file manually...")
!wget https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth -O /kaggle/working/efficientnet_b0_weights.pth
print("✅ Weights downloaded to /kaggle/working/efficientnet_b0_weights.pth")
print("You can now run the modified main cell (Step 2).")

--2025-11-28 17:29:57--  https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth
Resolving download.pytorch.org (download.pytorch.org)... 18.160.200.71, 18.160.200.126, 18.160.200.112, ...
Connecting to download.pytorch.org (download.pytorch.org)|18.160.200.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21444401 (20M) [binary/octet-stream]
Saving to: ‘/kaggle/working/efficientnet_b0_weights.pth’

/kaggle/working/eff 100%[===================>]  20.45M   133MB/s    in 0.2s    

2025-11-28 17:29:57 (133 MB/s) - ‘/kaggle/working/efficientnet_b0_weights.pth’ saved [21444401/21444401]

✅ Weights downloaded to /kaggle/working/efficientnet_b0_weights.pth
You can now run the modified main cell (Step 2).


In [ ]:

# =============================================================================
# TWO-STREAM EFFICIENTNET-B0 FOR DEEPFAKE DETECTION
# =============================================================================

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
from pathlib import Path
import numpy as np
import cv2
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import os # Import os to check for files

# =============================================================================
# DATASET WITH MOSAIC GENERATION (No changes)
# =============================================================================

class CelebDFWithMosaic(Dataset):
    """
    Dataset that returns:
    - 16 individual frames
    - 1 mosaic image (4x4 grid of all 16 frames)
    """
    def __init__(self, video_paths, labels, transform=None, frame_count=16):
        self.video_paths = video_paths
        self.labels = labels
        self.transform = transform
        self.frame_count = frame_count

    def __len__(self):
        return len(self.video_paths)

    def _sample_frames(self, video_path):
        cap = cv2.VideoCapture(str(video_path))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if not cap.isOpened() or total_frames < self.frame_count:
            if cap.isOpened(): cap.release()
            return np.zeros((self.frame_count, CONFIG['image_size'], CONFIG['image_size'], 3), dtype=np.uint8)
        indices = np.linspace(0, total_frames - 1, self.frame_count, dtype=int)
        frames = []
        for idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (CONFIG['image_size'], CONFIG['image_size']))
                frames.append(frame)
            else:
                frames.append(np.zeros((CONFIG['image_size'], CONFIG['image_size'], 3), dtype=np.uint8))
        cap.release()
        return np.array(frames)

    def _create_mosaic(self, frames):
        img_size = CONFIG['image_size']
        mosaic = np.zeros((img_size * 4, img_size * 4, 3), dtype=np.uint8)
        for idx, frame in enumerate(frames):
            row = idx // 4
            col = idx % 4
            mosaic[row*img_size:(row+1)*img_size, col*img_size:(col+1)*img_size] = frame
        return mosaic

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]
        frames_np = self._sample_frames(video_path)
        mosaic_np = self._create_mosaic(frames_np)
        if self.transform:
            frames = torch.stack([self.transform(frame) for frame in frames_np])
            mosaic = self.transform(mosaic_np)
        return frames, mosaic, label

# =============================================================================
# TWO-STREAM EFFICIENTNET-B0 MODEL (Manual Weight Loading)
# =============================================================================

class TwoStreamEfficientNet(nn.Module):
    def __init__(self, num_classes=2, pretrained=True):
        super().__init__()
        weights_path = "/kaggle/working/efficientnet_b0_weights.pth"
        efficientnet_temporal = models.efficientnet_b0(weights=None)
        efficientnet_spatial = models.efficientnet_b0(weights=None)
        
        if pretrained:
            print(f"Attempting to load weights from local file: {weights_path}")
            try:
                # Load onto the correct device
                state_dict = torch.load(weights_path, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
                efficientnet_temporal.load_state_dict(state_dict)
                efficientnet_spatial.load_state_dict(state_dict)
                print("✅ Weights loaded successfully from local file.")
            except Exception as e:
                print(f"🛑 FAILED to load weights: {e}")
            
        self.temporal_backbone = nn.Sequential(*list(efficientnet_temporal.children())[:-1])
        self.spatial_backbone = nn.Sequential(*list(efficientnet_spatial.children())[:-1])
        self.feature_dim = 1280
        self.temporal_lstm = nn.LSTM(self.feature_dim, 256, batch_first=True, bidirectional=True)
        self.spatial_pool = nn.AdaptiveAvgPool2d(1)
        self.fusion_classifier = nn.Sequential(
            nn.Linear(512 + self.feature_dim, 512),
            nn.ReLU(inplace=True),
            # Using the dropout value from your successful run
            nn.Dropout(0.6), 
            nn.Linear(512, num_classes)
        )

    def forward(self, frames, mosaic):
        batch_size = frames.size(0)
        num_frames = frames.size(1)
        frames_flat = frames.view(batch_size * num_frames, *frames.shape[2:])
        temporal_features = self.temporal_backbone(frames_flat)
        temporal_features = temporal_features.view(batch_size, num_frames, self.feature_dim)
        lstm_out, _ = self.temporal_lstm(temporal_features)
        temporal_out = lstm_out[:, -1, :]
        spatial_features = self.spatial_backbone(mosaic)
        spatial_features = self.spatial_pool(spatial_features)
        spatial_out = spatial_features.view(batch_size, self.feature_dim)
        fused_features = torch.cat([temporal_out, spatial_out], dim=1)
        output = self.fusion_classifier(fused_features)
        return fused_features, output

# =============================================================================
# PGD ADVERSARIAL ATTACK (Corrected)
# =============================================================================

def pgd_attack(model, frames, mosaic, labels, epsilon=4/255, alpha=1/255, num_iter=1): # Using num_iter=1
    frames_adv = frames.clone().detach()
    mosaic_adv = mosaic.clone().detach()
    frames_adv.requires_grad = True
    mosaic_adv.requires_grad = True
    criterion = nn.CrossEntropyLoss()
    
    for _ in range(num_iter):
        model.zero_grad()
        _, outputs = model(frames_adv, mosaic_adv)
        loss = criterion(outputs, labels)
        loss.backward()
        frames_adv = frames_adv.detach() + alpha * frames_adv.grad.sign()
        mosaic_adv = mosaic_adv.detach() + alpha * mosaic_adv.grad.sign()
        frames_perturbation = torch.clamp(frames_adv - frames, min=-epsilon, max=epsilon)
        mosaic_perturbation = torch.clamp(mosaic_adv - mosaic, min=-epsilon, max=epsilon)
        frames_adv = torch.clamp(frames + frames_perturbation, min=-10, max=10).detach()
        mosaic_adv = torch.clamp(mosaic + mosaic_perturbation, min=-10, max=10).detach()
        frames_adv.requires_grad = True
        mosaic_adv.requires_grad = True
    return frames_adv, mosaic_adv

# =============================================================================
# TRAINING WITH AUC TRACKING + AFSL (MODIFIED FOR CHECKPOINTING)
# =============================================================================

def train_with_auc(
    model, train_loader, val_loader, optimizer, scheduler, device, epochs, save_dir,
    # --- NEW: Parameters to allow resuming ---
    start_epoch,
    best_auc,
    patience_counter,
    history
):
    """Training loop with AFSL, AUC tracking, and Checkpointing."""
    save_dir.mkdir(parents=True, exist_ok=True)
    criterion = nn.CrossEntropyLoss()
    similarity_criterion = nn.CosineSimilarity(dim=1)
    
    # --- NEW: Define checkpoint paths ---
    best_model_path = save_dir / "best_model_two_stream.pth"
    resume_checkpoint_path = save_dir / "resume_checkpoint.pth"
    
    # AFSL hyperparameters (from your successful run)
    beta1 = 0.03
    beta2 = 0.003
    warmup_epochs = 3
    
    # --- MODIFIED: Start loop from 'start_epoch' ---
    for epoch in range(start_epoch, epochs):
        print(f"\n{'='*70}")
        print(f"Epoch {epoch + 1}/{epochs}") # Add +1 for 1-based display
        print(f"{'='*70}")
        
        use_afsl = epoch >= warmup_epochs
        if epoch == warmup_epochs and start_epoch <= warmup_epochs:
            print("🔥 Warmup complete! Activating AFSL...")
        
        # ===== TRAINING =====
        model.train()
        train_loss = 0.0
        all_preds_probs, all_labels = [], []
        
        for frames, mosaic, labels in tqdm(train_loader, desc="Training"):
            frames, mosaic, labels = frames.to(device), mosaic.to(device), labels.to(device)
            optimizer.zero_grad()
            clean_features, clean_outputs = model(frames, mosaic)
            loss_dcl = criterion(clean_outputs, labels)
            
            if use_afsl:
                frames_adv, mosaic_adv = pgd_attack(model, frames, mosaic, labels)
                adv_features, _ = model(frames_adv, mosaic_adv)
                loss_asl = (1 - similarity_criterion(clean_features, adv_features)).mean()
                real_mask, fake_mask = labels == 0, labels == 1
                loss_srl = 0.0
                if real_mask.sum() > 0 and fake_mask.sum() > 0:
                    real_features = clean_features[real_mask]
                    fake_features = clean_features[fake_mask]
                    loss_srl = similarity_criterion(
                        real_features.mean(dim=0, keepdim=True),
                        fake_features.mean(dim=0, keepdim=True)
                    ).mean()
                total_loss = loss_dcl + (beta1 * loss_asl) + (beta2 * loss_srl)
            else:
                total_loss = loss_dcl
            
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += total_loss.item() * frames.size(0)
            probs = torch.softmax(clean_outputs, dim=1)[:, 1]
            all_preds_probs.extend(probs.detach().cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
        try:
            train_auc = roc_auc_score(all_labels, all_preds_probs)
        except ValueError as e:
            print(f"Warning: Could not calculate train AUC. {e}")
            train_auc = 0.5
        avg_train_loss = train_loss / len(train_loader.dataset)
        
        # ===== VALIDATION =====
        model.eval()
        val_loss = 0.0
        val_preds_probs, val_labels = [], []
        
        with torch.no_grad():
            for frames, mosaic, labels in tqdm(val_loader, desc="Validation"):
                frames, mosaic, labels = frames.to(device), mosaic.to(device), labels.to(device)
                _, outputs = model(frames, mosaic)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * frames.size(0)
                probs = torch.softmax(outputs, dim=1)[:, 1]
                val_preds_probs.extend(probs.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        try:
            val_auc = roc_auc_score(val_labels, val_preds_probs)
        except ValueError as e:
            print(f"Warning: Could not calculate validation AUC. {e}")
            val_auc = 0.5
        avg_val_loss = val_loss / len(val_loader.dataset)
        
        history['train_auc'].append(train_auc)
        history['val_auc'].append(val_auc)
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        
        print(f"\n📊 Training:   Loss: {avg_train_loss:.4f} | AUC: {train_auc:.4f}")
        print(f"📊 Validation: Loss: {avg_val_loss:.4f} | AUC: {val_auc:.4f}")
        
        scheduler.step(val_auc)
        
        # Save best model
        if val_auc > best_auc:
            best_auc = val_auc
            patience_counter = 0
            torch.save({
                'epoch': epoch + 1, # Save as 1-based epoch
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_auc': val_auc,
            }, best_model_path)
            print(f"🎉 New best AUC: {best_auc:.4f} (Saved to {best_model_path})")
        else:
            patience_counter += 1
            print(f"⏳ Patience: {patience_counter}/{CONFIG['patience']}")
            if patience_counter >= CONFIG['patience']:
                print("🛑 Early stopping")
                break
        
        # --- NEW: Save a resume checkpoint EVERY epoch ---
        torch.save({
            'epoch': epoch + 1, # Save the *next* epoch to start from
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_auc': best_auc,
            'patience_counter': patience_counter,
            'history': history,
        }, resume_checkpoint_path)
        print(f"💾 Checkpoint saved to {resume_checkpoint_path} (can resume from epoch {epoch + 1})")
    
    return history, best_auc

# =============================================================================
# MAIN TRAINING FUNCTION (MODIFIED FOR CHECKPOINTING)
# =============================================================================

def main_two_stream():
    set_seed(CONFIG['seed'])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    real_videos, fake_videos = load_video_paths(CONFIG['dataset_path'])
    all_videos = real_videos + fake_videos
    all_labels = [0] * len(real_videos) + [1] * len(fake_videos)
    
    print(f"\n{'='*70}")
    print("TWO-STREAM EFFICIENTNET-B0 WITH MOSAIC (AFSL Corrected + Checkpointing)")
    print(f"{'='*70}")
    print(f"Dataset: {len(real_videos)} REAL, {len(fake_videos)} FAKE")
    print(f"Evaluation Metric: AUC (Area Under ROC Curve)\n")

    train_videos, val_videos, train_labels, val_labels = train_test_split(
        all_videos, all_labels, test_size=0.2, stratify=all_labels, random_state=CONFIG['seed']
    )

    # Adding the augmentations to fight overfitting
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomAffine(degrees=10, scale=(0.9, 1.1)), # Added
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.2, scale=(0.02, 0.1)), # Added
    ])
    val_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    train_dataset = CelebDFWithMosaic(train_videos, train_labels, transform=train_transform, frame_count=CONFIG['frame_count'])
    val_dataset = CelebDFWithMosaic(val_videos, val_labels, transform=val_transform, frame_count=CONFIG['frame_count'])

    class_counts = np.bincount(train_labels)
    class_weights = 1. / class_counts
    sample_weights = np.array([class_weights[label] for label in train_labels])
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
    
    # Make sure num_workers=0 in CONFIG cell!
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], sampler=sampler, num_workers=CONFIG['num_workers'], pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=CONFIG['num_workers'], pin_memory=True)

    # Model
    model = TwoStreamEfficientNet(pretrained=True).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
    
    # --- NEW: CHECKPOINT LOADING LOGIC ---
    start_epoch = 0
    best_auc = 0.0
    patience_counter = 0
    history = {'train_auc': [], 'val_auc': [], 'train_loss': [], 'val_loss': []}
    
    resume_checkpoint_path = CONFIG['save_dir'] / "resume_checkpoint.pth"

    if os.path.exists(resume_checkpoint_path):
        print(f"Resuming training from checkpoint: {resume_checkpoint_path}")
        checkpoint = torch.load(resume_checkpoint_path, map_location=device)
        
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
        start_epoch = checkpoint['epoch'] # Already saved as next epoch
        best_auc = checkpoint['best_auc']
        patience_counter = checkpoint['patience_counter']
        history = checkpoint['history']
        
        print(f"✅ Resumed successfully. Starting from epoch {start_epoch + 1}.") # +1 for display
    else:
        print("Starting a new training run.")
    # --- END NEW LOGIC ---

    print(f"\n🚀 Starting Training...")
    print(f"   Temporal Stream: 16 frames → EfficientNet-B0 → LSTM")
    print(f"   Spatial Stream: 4×4 mosaic → EfficientNet-B0")
    print(f"   Fusion: Concatenate → Classifier\n")
    
    start_time = time.time()
    history, best_auc = train_with_auc(
        model, train_loader, val_loader, optimizer, scheduler, device, CONFIG['epochs'], CONFIG['save_dir'],
        # Pass the new state variables
        start_epoch=start_epoch,
        best_auc=best_auc,
        patience_counter=patience_counter,
        history=history
    )
    end_time = time.time()
    
    print(f"\n{'='*70}")
    print(f"✅ Training Complete")
    print(f"{'='*70}")
    print(f"Time: {(end_time - start_time) / 60:.1f} minutes")
    print(f"Best Validation AUC: {best_auc:.4f}")

# This will run the training
main_two_stream()

Found 590 REAL videos and 5,639 FAKE videos.
Class distribution: REAL=9.5%, FAKE=90.5%

TWO-STREAM EFFICIENTNET-B0 WITH MOSAIC (AFSL Corrected + Checkpointing)
Dataset: 590 REAL, 5639 FAKE
Evaluation Metric: AUC (Area Under ROC Curve)

Attempting to load weights from local file: /kaggle/working/efficientnet_b0_weights.pth
✅ Weights loaded successfully from local file.
Starting a new training run.

🚀 Starting Training...
   Temporal Stream: 16 frames → EfficientNet-B0 → LSTM
   Spatial Stream: 4×4 mosaic → EfficientNet-B0
   Fusion: Concatenate → Classifier


Epoch 1/50


Validation: 100%|██████████| 312/312 [02:43<00:00,  1.91it/s]



📊 Training:   Loss: 0.6899 | AUC: 0.5977
📊 Validation: Loss: 0.5006 | AUC: 0.6126
🎉 New best AUC: 0.6126 (Saved to /kaggle/working/celeb_df_results_resnet_baseline/best_model_two_stream.pth)
💾 Checkpoint saved to /kaggle/working/celeb_df_results_resnet_baseline/resume_checkpoint.pth (can resume from epoch 1)

Epoch 2/50


Training:  70%|██████▉   | 866/1246 [08:22<03:10,  1.99it/s]

In [ ]:
# =============================================================================
# FINAL INFERENCE DEMO CELL
# 1. Run this cell.
# 2. Click "Choose Files" and upload a video.
# 3. The prediction will appear below.
# =============================================================================

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import numpy as np
import cv2
import time
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. Re-define all necessary classes and functions ---
# We must re-define the model architecture so PyTorch can load the weights.

# Ensure CONFIG is available, or define the necessary parts
try:
    IMAGE_SIZE = CONFIG['image_size']
    FRAME_COUNT = CONFIG['frame_count']
except NameError:
    print("CONFIG not found, using default values.")
    IMAGE_SIZE = 112
    FRAME_COUNT = 16

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TwoStreamEfficientNet(nn.Module):
    """
    Two-stream architecture (re-defined from main script).
    """
    def __init__(self, num_classes=2, pretrained=True):
        super().__init__()
        
        weights_path = "/kaggle/working/efficientnet_b0_weights.pth"
        
        efficientnet_temporal = models.efficientnet_b0(weights=None)
        efficientnet_spatial = models.efficientnet_b0(weights=None)
        
        if pretrained:
            print(f"Loading pre-trained weights from local file: {weights_path}")
            try:
                state_dict = torch.load(weights_path, map_location=DEVICE)
                efficientnet_temporal.load_state_dict(state_dict)
                efficientnet_spatial.load_state_dict(state_dict)
                print("✅ EfficientNet backbones loaded.")
            except Exception as e:
                print(f"🛑 FAILED to load weights: {e}")
        
        self.temporal_backbone = nn.Sequential(*list(efficientnet_temporal.children())[:-1])
        self.spatial_backbone = nn.Sequential(*list(efficientnet_spatial.children())[:-1])
        self.feature_dim = 1280
        self.temporal_lstm = nn.LSTM(self.feature_dim, 256, batch_first=True, bidirectional=True)
        self.spatial_pool = nn.AdaptiveAvgPool2d(1)
        self.fusion_classifier = nn.Sequential(
            nn.Linear(512 + self.feature_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.6), # Using the 0.6 dropout from our best model
            nn.Linear(512, num_classes)
        )

    def forward(self, frames, mosaic):
        batch_size = frames.size(0)
        num_frames = frames.size(1)
        
        frames_flat = frames.view(batch_size * num_frames, *frames.shape[2:])
        temporal_features = self.temporal_backbone(frames_flat)
        temporal_features = temporal_features.view(batch_size, num_frames, self.feature_dim)
        lstm_out, _ = self.temporal_lstm(temporal_features)
        temporal_out = lstm_out[:, -1, :]
        
        spatial_features = self.spatial_backbone(mosaic)
        spatial_features = self.spatial_pool(spatial_features)
        spatial_out = spatial_features.view(batch_size, self.feature_dim)
        
        fused_features = torch.cat([temporal_out, spatial_out], dim=1)
        output = self.fusion_classifier(fused_features)
        
        return fused_features, output

# --- 2. Re-define preprocessing functions ---

def _sample_frames(video_path):
    cap = cv2.VideoCapture(str(video_path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if not cap.isOpened() or total_frames < FRAME_COUNT:
        if cap.isOpened(): cap.release()
        print(f"Error: Video {video_path} could not be opened or has < {FRAME_COUNT} frames.")
        return None

    indices = np.linspace(0, total_frames - 1, FRAME_COUNT, dtype=int)
    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (IMAGE_SIZE, IMAGE_SIZE))
            frames.append(frame)
        else:
            frames.append(np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8))
    cap.release()
    return np.array(frames)

def _create_mosaic(frames):
    mosaic = np.zeros((IMAGE_SIZE * 4, IMAGE_SIZE * 4, 3), dtype=np.uint8)
    for idx, frame in enumerate(frames):
        row = idx // 4
        col = idx % 4
        mosaic[row*IMAGE_SIZE:(row+1)*IMAGE_SIZE, col*IMAGE_SIZE:(col+1)*IMAGE_SIZE] = frame
    return mosaic

# Define the *validation* transform (no augmentations)
val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# --- 3. Load Our Trained Model ---

print("\n" + "="*70)
print("LOADING TRAINED DEEPFAKE DETECTOR...")
print("="*70)

# Initialize model
model = TwoStreamEfficientNet(pretrained=True).to(DEVICE)

# Load the saved weights from our best run
MODEL_PATH = "/kaggle/working/celeb_df_results_resnet_baseline/best_model_two_stream.pth"
try:
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✅ Successfully loaded best model weights from epoch {checkpoint['epoch']}")
    print(f"   (Achieved {checkpoint['val_auc']:.4f} AUC during training)")
except Exception as e:
    print(f"🛑 FAILED to load trained model weights from {MODEL_PATH}")
    print(f"   Error: {e}")
    print("   Please ensure you have run the training cell first!")

model.eval() # Set model to evaluation mode

# --- 4. Define the Inference Function ---

def predict_video(video_path):
    """
    Takes a path to a single video, preprocesses it, and runs inference.
    """
    # Preprocess the video
    frames_np = _sample_frames(video_path)
    if frames_np is None:
        return "Error", 0.0, "Could not process video."

    mosaic_np = _create_mosaic(frames_np)
    
    # Apply transforms
    frames = torch.stack([val_transform(frame) for frame in frames_np])
    mosaic = val_transform(mosaic_np)
    
    # Add batch dimension (batch size = 1)
    frames = frames.unsqueeze(0).to(DEVICE)
    mosaic = mosaic.unsqueeze(0).to(DEVICE)
    
    # Run inference
    with torch.no_grad():
        _, outputs = model(frames, mosaic)
        
        # Get probabilities
        probabilities = torch.softmax(outputs, dim=1)
        
        # Get confidence and prediction
        # [0] is REAL, [1] is FAKE
        confidence_fake = probabilities[0][1].item()
        confidence_real = probabilities[0][0].item()
        
        if confidence_fake > confidence_real:
            prediction = "FAKE"
            confidence = confidence_fake
            caption = f"The model is {confidence*100:.2f}% sure this video is FAKE."
        else:
            prediction = "REAL"
            confidence = confidence_real
            caption = f"The model is {confidence*100:.2f}% sure this video is REAL."
            
        return prediction, confidence, caption

# --- 5. Create the UI (ipywidgets) ---

# Output widget to display results
out = widgets.Output(layout={'border': '1px solid black', 'padding': '10px'})

# File uploader widget
uploader = widgets.FileUpload(
    accept='video/*',
    multiple=False,
    description='Upload Video'
)

# Event handler for when a file is uploaded
def on_upload_change(change):
    with out:
        clear_output(wait=True) # Clear previous results
        print("File uploaded. Processing...")
        
        # Get uploaded file data
        uploaded_file = list(change['new'].values())[0]
        content = uploaded_file['content']
        name = uploaded_file['name']
        
        # Save temporary file to disk
        temp_path = f"/kaggle/working/{name}"
        with open(temp_path, 'wb') as f:
            f.write(content)
            
        print(f"Running detection on {name}...")
        
        # Run prediction
        start_time = time.time()
        prediction, confidence, caption = predict_video(temp_path)
        end_time = time.time()
        
        # Print results
        print("\n" + "="*30)
        print("  DETECTION RESULTS")
        print("="*30)
        print(f"  Prediction: {prediction}")
        print(f"  Confidence: {confidence*100:.2f}%")
        print(f"  Analysis:   {caption}")
        print(f"  (Time taken: {end_time - start_time:.2f} seconds)")

# Link the event handler to the uploader
uploader.observe(on_upload_change, names='value')

# Display the UI
print("\n" + "="*70)
print("  LIVE DEEPFAKE DETECTOR")
print("="*70)
print("Upload a video file to run a real-time prediction.")
display(uploader, out)

In [ ]:
# =============================================================================
# FINAL INFERENCE DEMO CELL (PERMANENT)
#
# INSTRUCTIONS:
# 1. DO NOT run the 6.5-hour training cell.
# 2. Add your saved Kaggle Model (e.g., "my-deepfake-detector-v1")
#    using the "Add Data" button in the sidebar.
# 3. Update the `MODEL_INPUT_PATH` variable below to match the name.
# 4. Run this cell.
# =============================================================================

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import numpy as np
import cv2
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
import os # Import os to check paths

# --- 1. SET YOUR PERMANENT MODEL PATH HERE ---
# Find this path in your /kaggle/input/ directory after adding your model
MODEL_INPUT_PATH = "/kaggle/input/my-deepfake-detector-v1/" 
# -----------------------------------------------

# Define file paths
WEIGHTS_FILE = "efficientnet_b0_weights.pth"
MODEL_FILE = "best_model_two_stream.pth" # This file is inside the "celeb_df_results_resnet_baseline" folder
                                         # If you saved it from there, the path might be:
                                         # MODEL_FILE = "celeb_df_results_resnet_baseline/best_model_two_stream.pth"
                                         # Please check your /kaggle/input/my-deepfake-detector-v1/ structure

# Check if the path is correct
if not os.path.exists(MODEL_INPUT_PATH):
    print(f"🛑 ERROR: The path '{MODEL_INPUT_PATH}' does not exist.")
    print("Please check the 'Add Data' sidebar and update the MODEL_INPUT_PATH variable.")
else:
    print(f"✅ Found model directory: {MODEL_INPUT_PATH}")


# --- 2. Re-define all necessary classes and functions ---
try:
    IMAGE_SIZE = CONFIG['image_size']
    FRAME_COUNT = CONFIG['frame_count']
except NameError:
    print("CONFIG not found, using default values.")
    IMAGE_SIZE = 112
    FRAME_COUNT = 16

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TwoStreamEfficientNet(nn.Module):
    def __init__(self, num_classes=2, pretrained=True):
        super().__init__()
        
        weights_path = os.path.join(MODEL_INPUT_PATH, WEIGHTS_FILE)
        
        efficientnet_temporal = models.efficientnet_b0(weights=None)
        efficientnet_spatial = models.efficientnet_b0(weights=None)
        
        if pretrained:
            print(f"Loading pre-trained weights from local file: {weights_path}")
            try:
                state_dict = torch.load(weights_path, map_location=DEVICE)
                efficientnet_temporal.load_state_dict(state_dict)
                efficientnet_spatial.load_state_dict(state_dict)
                print("✅ EfficientNet backbones loaded.")
            except Exception as e:
                print(f"🛑 FAILED to load weights: {e}")
        
        self.temporal_backbone = nn.Sequential(*list(efficientnet_temporal.children())[:-1])
        self.spatial_backbone = nn.Sequential(*list(efficientnet_spatial.children())[:-1])
        self.feature_dim = 1280
        self.temporal_lstm = nn.LSTM(self.feature_dim, 256, batch_first=True, bidirectional=True)
        self.spatial_pool = nn.AdaptiveAvgPool2d(1)
        self.fusion_classifier = nn.Sequential(
            nn.Linear(512 + self.feature_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.6),
            nn.Linear(512, num_classes)
        )

    def forward(self, frames, mosaic):
        batch_size = frames.size(0)
        num_frames = frames.size(1)
        frames_flat = frames.view(batch_size * num_frames, *frames.shape[2:])
        temporal_features = self.temporal_backbone(frames_flat)
        temporal_features = temporal_features.view(batch_size, num_frames, self.feature_dim)
        lstm_out, _ = self.temporal_lstm(temporal_features)
        temporal_out = lstm_out[:, -1, :]
        spatial_features = self.spatial_backbone(mosaic)
        spatial_features = self.spatial_pool(spatial_features)
        spatial_out = spatial_features.view(batch_size, self.feature_dim)
        fused_features = torch.cat([temporal_out, spatial_out], dim=1)
        output = self.fusion_classifier(fused_features)
        return fused_features, output

# --- 3. Re-define preprocessing functions ---

def _sample_frames(video_path):
    cap = cv2.VideoCapture(str(video_path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if not cap.isOpened() or total_frames < FRAME_COUNT:
        if cap.isOpened(): cap.release()
        print(f"Error: Video {video_path} could not be opened or has < {FRAME_COUNT} frames.")
        return None
    indices = np.linspace(0, total_frames - 1, FRAME_COUNT, dtype=int)
    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (IMAGE_SIZE, IMAGE_SIZE))
            frames.append(frame)
        else:
            frames.append(np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.uint8))
    cap.release()
    return np.array(frames)

def _create_mosaic(frames):
    mosaic = np.zeros((IMAGE_SIZE * 4, IMAGE_SIZE * 4, 3), dtype=np.uint8)
    for idx, frame in enumerate(frames):
        row = idx // 4
        col = idx % 4
        mosaic[row*IMAGE_SIZE:(row+1)*IMAGE_SIZE, col*IMAGE_SIZE:(col+1)*IMAGE_SIZE] = frame
    return mosaic

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# --- 4. Load Our Trained Model ---

print("\n" + "="*70)
print("LOADING TRAINED DEEPFAKE DETECTOR...")
print("="*70)

model = TwoStreamEfficientNet(pretrained=True).to(DEVICE)

# --- This is the new, permanent path ---
MODEL_PATH = os.path.join(MODEL_INPUT_PATH, MODEL_FILE)
# --------------------------------------

try:
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✅ Successfully loaded best model weights from epoch {checkpoint['epoch']}")
    print(f"   (Achieved {checkpoint['val_auc']:.4f} AUC during training)")
except Exception as e:
    print(f"🛑 FAILED to load trained model weights from {MODEL_PATH}")
    print(f"   Error: {e}")
    print("   Please ensure the `MODEL_INPUT_PATH` is correct and you saved the model.")

model.eval()

# --- 5. Define the Inference Function ---

def predict_video(video_path):
    frames_np = _sample_frames(video_path)
    if frames_np is None:
        return "Error", 0.0, "Could not process video."
    mosaic_np = _create_mosaic(frames_np)
    frames = torch.stack([val_transform(frame) for frame in frames_np])
    mosaic = val_transform(mosaic_np)
    frames = frames.unsqueeze(0).to(DEVICE)
    mosaic = mosaic.unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        _, outputs = model(frames, mosaic)
        probabilities = torch.softmax(outputs, dim=1)
        confidence_fake = probabilities[0][1].item()
        confidence_real = probabilities[0][0].item()
        
        if confidence_fake > confidence_real:
            prediction = "FAKE"
            confidence = confidence_fake
            caption = f"The model is {confidence*100:.2f}% sure this video is FAKE."
        else:
            prediction = "REAL"
            confidence = confidence_real
            caption = f"The model is {confidence*100:.2f}% sure this video is REAL."
            
        return prediction, confidence, caption

# --- 6. Create the UI (ipywidgets) ---

out = widgets.Output(layout={'border': '1px solid black', 'padding': '10px'})
uploader = widgets.FileUpload(accept='video/*', multiple=False, description='Upload Video')

def on_upload_change(change):
    with out:
        clear_output(wait=True)
        print("File uploaded. Processing...")
        
        uploaded_file = list(change['new'].values())[0]
        content = uploaded_file['content']
        name = uploaded_file['name']
        
        # Save temporary file to /kaggle/working/
        temp_path = f"/kaggle/working/{name}"
        with open(temp_path, 'wb') as f:
            f.write(content)
            
        print(f"Running detection on {name}...")
        
        start_time = time.time()
        prediction, confidence, caption = predict_video(temp_path)
        end_time = time.time()
        
        print("\n" + "="*30)
        print("  DETECTION RESULTS")
        print("="*30)
        print(f"  Prediction: {prediction}")
        print(f"  Confidence: {confidence*100:.2f}%")
        print(f"  Analysis:   {caption}")
        print(f"  (Time taken: {end_time - start_time:.2f} seconds)")

uploader.observe(on_upload_change, names='value')

print("\n" + "="*70)
print("  LIVE DEEPFAKE DETECTOR")
print("="*70)
print("Upload a video file to run a real-time prediction.")
display(uploader, out)